In [1]:
%load_ext autoreload
%autoreload

import os
import sys
import random
import django
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from datetime import datetime
import time
from collections import Counter
import pandas as pd
import numpy as np
import random
from tqdm import tqdm_notebook

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.apps import apps
from django_currentuser.middleware import get_current_user
from django.db.models import F, Q, Sum, Count, Case, When, IntegerField, Value

In [2]:
bgr = m.BalancegameRecord.objects.filter(who=638).exclude(tag_pos__isnull=True).select_related('tag_pos', 'tag_neg').order_by('-id')[:100]

flds = [
    'type','category','item',
    'color','detail','pattern','texture','look','length','sleeve_length',
    'neckline','fit','shape','heel_height','heel_shape','toe_type','sole_type',
    'strap','size','main_material','sub_material'
]

df_pos = pd.DataFrame([r.tag_pos.__dict__ for r in bgr])[flds]
df_neg = pd.DataFrame([r.tag_neg.__dict__ for r in bgr])[flds]

In [3]:
def get_baseframe(df0):
    _df = df0.set_index(['type','category','item']).stack().to_frame()
    _df[1] = 1
    _df.index.names = ['type','category','item','prop']
    _df.columns = ['val', 'count']
    return _df.groupby(by=['type','category','item','prop','val']).count()

In [4]:
df_pos = get_baseframe(df_pos)
df_neg = get_baseframe(df_neg)

In [5]:
# df = df_pos - df_neg
df = df_pos.sub(df_neg, fill_value=0)
df = df[df['count'] > 0]
df#['count'].sum()

count
type        category item prop        val          
accessories 잡화       머플러  color       스카이블루     1.0
                          detail      프린지       1.0
                          texture     울/캐시미어    1.0
                     선글라스 color       블랙        1.0
                          texture     메탈        1.0
...                                             ...
shoes       펌프스      펌프스  heel_height 플랫        1.0
                          heel_shape  두꺼운굽      1.0
                          sole_type   플랫폼       1.0
                          toe_type    라운드토      1.0
                                      스퀘어토      1.0

[363 rows x 1 columns]

In [109]:
df.reset_index().to_dict('records')

[{'type': 'accessories',
  'category': '잡화',
  'item': '손목시계',
  'prop': 'texture',
  'val': '가죽',
  'count': 1.0},
 {'type': 'clothing',
  'category': '니트웨어',
  'item': '가디건',
  'prop': 'neckline',
  'val': '노카라',
  'count': 1.0},
 {'type': 'clothing',
  'category': '니트웨어',
  'item': '가디건',
  'prop': 'texture',
  'val': '니트',
  'count': 1.0},
 {'type': 'clothing',
  'category': '베스트',
  'item': '베스트',
  'prop': 'fit',
  'val': '노멀',
  'count': 2.0},
 {'type': 'clothing',
  'category': '베스트',
  'item': '베스트',
  'prop': 'length',
  'val': '노멀',
  'count': 2.0},
 {'type': 'clothing',
  'category': '베스트',
  'item': '베스트',
  'prop': 'look',
  'val': '캐주얼',
  'count': 2.0},
 {'type': 'clothing',
  'category': '베스트',
  'item': '베스트',
  'prop': 'neckline',
  'val': '브이넥',
  'count': 1.0},
 {'type': 'clothing',
  'category': '베스트',
  'item': '베스트',
  'prop': 'pattern',
  'val': '무지',
  'count': 1.0},
 {'type': 'clothing',
  'category': '베스트',
  'item': '베스트',
  'prop': 'sleeve_length',
  'val'

In [28]:
df['count'].unstack()

val                                    스카이블루  프린지  울/캐시미어   블랙   메탈  앱스트랙트  \
type        category item prop                                               
accessories 잡화       머플러  color          1.0  NaN     NaN  NaN  NaN    NaN   
                          detail         NaN  1.0     NaN  NaN  NaN    NaN   
                          texture        NaN  NaN     1.0  NaN  NaN    NaN   
                     선글라스 color          NaN  NaN     NaN  1.0  NaN    NaN   
                          texture        NaN  NaN     NaN  NaN  1.0    NaN   
...                                      ...  ...     ...  ...  ...    ...   
shoes       펌프스      펌프스  detail         NaN  NaN     NaN  NaN  NaN    NaN   
                          heel_height    NaN  NaN     NaN  NaN  NaN    NaN   
                          heel_shape     NaN  NaN     NaN  NaN  NaN    NaN   
                          sole_type      NaN  NaN     NaN  NaN  NaN    NaN   
                          toe_type       NaN  NaN     NaN  NaN  NaN    NaN   

val                                    스트랩  플라스틱/PVC  지퍼  그래픽  ...  원형굽  레이스업  \
type        category item prop                                 ...              
accessories 잡화       머플러  color        NaN       NaN NaN  NaN  ...  NaN   NaN   
                          detail       NaN       NaN NaN  NaN  ...  NaN   NaN   
                          texture      NaN       NaN NaN  NaN  ...  NaN   NaN   
                     선글라스 color        NaN       NaN NaN  NaN  ...  NaN   NaN   
                          texture      NaN       NaN NaN  NaN  ...  NaN   NaN   
...                                    ...       ...  ..  ...  ...  ...   ...   
shoes       펌프스      펌프스  detail       NaN       NaN NaN  NaN  ...  NaN   NaN   
                          heel_height  NaN       NaN NaN  NaN  ...  NaN   NaN   
                          heel_shape   NaN       NaN NaN  NaN  ...  NaN   NaN   
                          sole_type    NaN       NaN NaN  NaN  ...  NaN   NaN   
                          toe_type     NaN       NaN NaN  NaN  ...  NaN   NaN   

val                                    라운드토  앵클스트랩   플랫  두꺼운굽   퍼  오픈토  로고  \
type        category item prop                                               
accessories 잡화       머플러  color         NaN    NaN  NaN   NaN NaN  NaN NaN   
                          detail        NaN    NaN  NaN   NaN NaN  NaN NaN   
                          texture       NaN    NaN  NaN   NaN NaN  NaN NaN   
                     선글라스 color         NaN    NaN  NaN   NaN NaN  NaN NaN   
                          texture       NaN    NaN  NaN   NaN NaN  NaN NaN   
...                                     ...    ...  ...   ...  ..  ...  ..   
shoes       펌프스      펌프스  detail        NaN    NaN  NaN   NaN NaN  NaN NaN   
                          heel_height   NaN    NaN  1.0   NaN NaN  NaN NaN   
                          heel_shape    NaN    NaN  NaN   1.0 NaN  NaN NaN   
                          sole_type     NaN    NaN  NaN   NaN NaN  NaN NaN   
                          toe_type      1.0    NaN  NaN   NaN NaN  NaN NaN   

val                                    스퀘어토  
type        category item prop               
accessories 잡화       머플러  color         NaN  
                          detail        NaN  
                          texture       NaN  
                     선글라스 color         NaN  
                          texture       NaN  
...                                     ...  
shoes       펌프스      펌프스  detail        NaN  
                          heel_height   NaN  
                          heel_shape    NaN  
                          sole_type     NaN  
                          toe_type      1.0  

[311 rows x 100 columns]

In [29]:
df['count'].unstack().loc['accessories']

val                    스카이블루  프린지  울/캐시미어   블랙   메탈  앱스트랙트  스트랩  플라스틱/PVC  지퍼  \
category item prop                                                              
잡화       머플러  color      1.0  NaN     NaN  NaN  NaN    NaN  NaN       NaN NaN   
              detail     NaN  1.0     NaN  NaN  NaN    NaN  NaN       NaN NaN   
              texture    NaN  NaN     1.0  NaN  NaN    NaN  NaN       NaN NaN   
         선글라스 color      NaN  NaN     NaN  1.0  NaN    NaN  NaN       NaN NaN   
              texture    NaN  NaN     NaN  NaN  1.0    NaN  NaN       NaN NaN   
         스카프  pattern    NaN  NaN     NaN  NaN  NaN    1.0  NaN       NaN NaN   
         안경   detail     NaN  NaN     NaN  NaN  NaN    NaN  1.0       NaN NaN   
              texture    NaN  NaN     NaN  NaN  NaN    NaN  NaN       1.0 NaN   

val                    그래픽  ...  원형굽  레이스업  라운드토  앵클스트랩  플랫  두꺼운굽   퍼  오픈토  \
category item prop          ...                                              
잡화       머플러  color    NaN  ...  NaN   NaN   NaN    NaN NaN   NaN NaN  NaN   
              detail   NaN  ...  NaN   NaN   NaN    NaN NaN   NaN NaN  NaN   
              texture  NaN  ...  NaN   NaN   NaN    NaN NaN   NaN NaN  NaN   
         선글라스 color    NaN  ...  NaN   NaN   NaN    NaN NaN   NaN NaN  NaN   
              texture  NaN  ...  NaN   NaN   NaN    NaN NaN   NaN NaN  NaN   
         스카프  pattern  NaN  ...  NaN   NaN   NaN    NaN NaN   NaN NaN  NaN   
         안경   detail   NaN  ...  NaN   NaN   NaN    NaN NaN   NaN NaN  NaN   
              texture  NaN  ...  NaN   NaN   NaN    NaN NaN   NaN NaN  NaN   

val                    로고  스퀘어토  
category item prop               
잡화       머플러  color   NaN   NaN  
              detail  NaN   NaN  
              texture NaN   NaN  
         선글라스 color   NaN   NaN  
              texture NaN   NaN  
         스카프  pattern NaN   NaN  
         안경   detail  NaN   NaN  
              texture NaN   NaN  

[8 rows x 100 columns]

In [30]:
df['count'].unstack().loc['accessories'].to_excel('test_count.xlsx')

In [25]:
maxsub = df['count'].unstack().idxmax(axis=1)#.to_frame()
maxsub

type         category  item  prop       
accessories  잡화        머플러   color           스카이블루
                             detail            프린지
                             texture        울/캐시미어
                       선글라스  color              블랙
                             texture            메탈
                                             ...  
shoes        펌프스       펌프스   detail             판넬
                             heel_height        플랫
                             heel_shape       두꺼운굽
                             sole_type         플랫폼
                             toe_type         라운드토
Length: 311, dtype: object

In [26]:
maxprop = df['count'].unstack().groupby(by=['type','prop']).count().idxmax(axis=1)#.to_frame()
maxprop.name = 'val'
maxprop.reset_index().to_dict('records')

[{'type': 'accessories', 'prop': 'color', 'val': '스카이블루'},
 {'type': 'accessories', 'prop': 'detail', 'val': '프린지'},
 {'type': 'accessories', 'prop': 'pattern', 'val': '앱스트랙트'},
 {'type': 'accessories', 'prop': 'texture', 'val': '울/캐시미어'},
 {'type': 'bags', 'prop': 'color', 'val': '블랙'},
 {'type': 'bags', 'prop': 'detail', 'val': '지퍼'},
 {'type': 'bags', 'prop': 'pattern', 'val': '그래픽'},
 {'type': 'bags', 'prop': 'shape', 'val': '사각형'},
 {'type': 'bags', 'prop': 'size', 'val': '라지'},
 {'type': 'bags', 'prop': 'strap', 'val': False},
 {'type': 'bags', 'prop': 'texture', 'val': '가죽'},
 {'type': 'clothing', 'prop': 'color', 'val': '블랙'},
 {'type': 'clothing', 'prop': 'detail', 'val': '포켓'},
 {'type': 'clothing', 'prop': 'fit', 'val': '노멀'},
 {'type': 'clothing', 'prop': 'length', 'val': '롱'},
 {'type': 'clothing', 'prop': 'look', 'val': '캐주얼'},
 {'type': 'clothing', 'prop': 'neckline', 'val': '셔츠카라'},
 {'type': 'clothing', 'prop': 'pattern', 'val': '무지'},
 {'type': 'clothing', 'prop': 'sh

In [31]:
maxprop_total = df['count'].unstack().groupby(by=['prop']).count().idxmax(axis=1)#.to_frame()
maxprop_total.name = 'val'
maxprop_total.reset_index().to_dict('records')

[{'prop': 'color', 'val': '블랙'},
 {'prop': 'detail', 'val': '포켓'},
 {'prop': 'fit', 'val': '노멀'},
 {'prop': 'heel_height', 'val': '플랫'},
 {'prop': 'heel_shape', 'val': '두꺼운굽'},
 {'prop': 'length', 'val': '롱'},
 {'prop': 'look', 'val': '캐주얼'},
 {'prop': 'neckline', 'val': '셔츠카라'},
 {'prop': 'pattern', 'val': '무지'},
 {'prop': 'shape', 'val': 'A라인'},
 {'prop': 'size', 'val': '라지'},
 {'prop': 'sleeve_length', 'val': '긴팔'},
 {'prop': 'sole_type', 'val': '플랫폼'},
 {'prop': 'strap', 'val': False},
 {'prop': 'texture', 'val': '합성섬유'},
 {'prop': 'toe_type', 'val': '라운드토'}]

In [2]:
%%time
bgr = m.BalancegameRecord.objects.exclude(tag_pos__isnull=True).select_related('tag_pos', 'tag_neg')

df_pos = pd.DataFrame([r.tag_pos.__dict__ for r in bgr])
df_neg = pd.DataFrame([r.tag_neg.__dict__ for r in bgr])

flds = [
    'type','category','item',
    'color','detail','pattern','texture','look','length','sleeve_length',
    'neckline','fit','shape','heel_height','heel_shape','toe_type','sole_type',
    'strap','size','main_material','sub_material'
]

count = {col:Counter(df_pos[col])-Counter(df_neg[col]) for col in flds }; count

CPU times: user 51.2 ms, sys: 4.36 ms, total: 55.5 ms
Wall time: 2.09 s


{'type': Counter(),
 'category': Counter({'운동화': 7, '목걸이': 2}),
 'item': Counter({'컨버스': 1,
          '청바지': 2,
          '슬리퍼': 1,
          '트렌치코트': 1,
          '버킷햇': 1,
          '모피코트': 2,
          '야상': 5,
          '에코백': 2,
          '런닝화': 5,
          '무스탕': 3,
          '스커트': 3,
          '팔찌/발찌': 2,
          '재킷': 7,
          '후드': 5,
          '탱크탑': 1,
          '스웨트팬츠': 5,
          '스웨터': 5,
          '목걸이': 2,
          '토트백': 2,
          '베스트': 3,
          '플리플랍': 2,
          '플리츠스커트': 3,
          '슬립드레스': 1,
          '뷔스티에': 2,
          '클러치': 1,
          '니트베스트': 1,
          '더플코트': 1,
          '카고팬츠': 1,
          '하이탑스니커즈': 1,
          '데님재킷': 3,
          '튜닉원피스': 2,
          '배기팬츠': 1,
          '바람막이': 6,
          '캡': 1,
          '와이드팬츠': 3,
          '벨트백': 1,
          '터틀넥': 2,
          '펌프스': 2,
          '니트원피스': 2,
          '홀터넥드레스': 3,
          '원피스수영복': 1,
          '워커': 2,
          '패딩조끼': 2,
          '와이셔츠': 1,
          '멜빵원피

In [3]:
dfdf_pos = df_pos[flds].set_index(['type','category','item']).stack().to_frame()
dfdf_pos[1] = 1
dfdf_pos.index.names = ['type','category','item','sub']
dfdf_pos.columns = ['val', 'count']
dfdf_pos = dfdf_pos.groupby(by=['type','category','item','sub','val']).count()

In [4]:
dfdf_neg = df_neg[flds].set_index(['type','category','item']).stack().to_frame()
dfdf_neg[1] = 1
dfdf_neg.index.names = ['type','category','item','sub']
dfdf_neg.columns = ['val', 'count']
dfdf_neg = dfdf_neg.groupby(by=['type','category','item','sub','val']).count()

In [5]:
dfdf = dfdf_pos - dfdf_neg
dfdf = dfdf[dfdf['count']>0]
dfdf

count
type        category item sub      val         
accessories 잡화       벨트   detail   버클       1.0
                     선글라스 color    블랙       2.0
                     손목시계 color    블랙       2.0
                          pattern  무지       1.0
                          texture  메탈       1.0
...                                         ...
shoes       펌프스      펌프스  strap    False    2.0
                          texture  가죽       1.0
                                   스웨이드     1.0
                          toe_type 오픈토      3.0
                                   포인티토     4.0

[430 rows x 1 columns]

In [6]:
dfdf['count'].unstack()

val                                     버클   블랙   무지   메탈  보우/타이  앱스트랙트  \
type        category item sub                                             
accessories 잡화       벨트   detail       1.0  NaN  NaN  NaN    NaN    NaN   
                     선글라스 color        NaN  2.0  NaN  NaN    NaN    NaN   
                     손목시계 color        NaN  2.0  NaN  NaN    NaN    NaN   
                          pattern      NaN  NaN  1.0  NaN    NaN    NaN   
                          texture      NaN  NaN  NaN  1.0    NaN    NaN   
...                                    ...  ...  ...  ...    ...    ...   
shoes       펌프스      펌프스  strap        NaN  NaN  NaN  NaN    NaN    NaN   
                          heel_height  NaN  NaN  NaN  NaN    NaN    NaN   
                          toe_type     NaN  NaN  NaN  NaN    NaN    NaN   
                          heel_shape   NaN  NaN  NaN  NaN    NaN    NaN   
                          sole_type    NaN  NaN  NaN  NaN    NaN    NaN   

val                                    플라스틱/PVC  브라운  화이트  밴딩  ...  원형굽  플랫폼  \
type        category item sub                                  ...             
accessories 잡화       벨트   detail            NaN  NaN  NaN NaN  ...  NaN  NaN   
                     선글라스 color             NaN  NaN  NaN NaN  ...  NaN  NaN   
                     손목시계 color             NaN  NaN  NaN NaN  ...  NaN  NaN   
                          pattern           NaN  NaN  NaN NaN  ...  NaN  NaN   
                          texture           NaN  NaN  NaN NaN  ...  NaN  NaN   
...                                         ...  ...  ...  ..  ...  ...  ...   
shoes       펌프스      펌프스  strap             NaN  NaN  NaN NaN  ...  NaN  NaN   
                          heel_height       NaN  NaN  NaN NaN  ...  NaN  NaN   
                          toe_type          NaN  NaN  NaN NaN  ...  NaN  NaN   
                          heel_shape        NaN  NaN  NaN NaN  ...  NaN  NaN   
                          sole_type         NaN  NaN  NaN NaN  ...  NaN  1.0   

val                                    스틸레토굽  앵클스트랩  고무  오픈토  아몬드토  하이힐   웨지  \
type        category item sub                                                  
accessories 잡화       벨트   detail         NaN    NaN NaN  NaN   NaN  NaN  NaN   
                     선글라스 color          NaN    NaN NaN  NaN   NaN  NaN  NaN   
                     손목시계 color          NaN    NaN NaN  NaN   NaN  NaN  NaN   
                          pattern        NaN    NaN NaN  NaN   NaN  NaN  NaN   
                          texture        NaN    NaN NaN  NaN   NaN  NaN  NaN   
...                                      ...    ...  ..  ...   ...  ...  ...   
shoes       펌프스      펌프스  strap          NaN    NaN NaN  NaN   NaN  NaN  NaN   
                          heel_height    NaN    NaN NaN  NaN   NaN  2.0  NaN   
                          toe_type       NaN    NaN NaN  3.0   NaN  NaN  NaN   
                          heel_shape     3.0    NaN NaN  NaN   NaN  NaN  NaN   
                          sole_type      NaN    NaN NaN  NaN   NaN  NaN  1.0   

val                                    포인티토  
type        category item sub                
accessories 잡화       벨트   detail        NaN  
                     선글라스 color         NaN  
                     손목시계 color         NaN  
                          pattern       NaN  
                          texture       NaN  
...                                     ...  
shoes       펌프스      펌프스  strap         NaN  
                          heel_height   NaN  
                          toe_type      4.0  
                          heel_shape    NaN  
                          sole_type     NaN  

[371 rows x 97 columns]

In [7]:
maxsub = dfdf['count'].unstack().idxmax(axis=1).to_frame()
maxsub

0
type        category item sub               
accessories 잡화       벨트   detail          버클
                     선글라스 color           블랙
                     손목시계 color           블랙
                          pattern         무지
                          texture         메탈
...                                      ...
shoes       펌프스      펌프스  strap        False
                          heel_height    하이힐
                          toe_type      포인티토
                          heel_shape   스틸레토굽
                          sole_type      플랫폼

[371 rows x 1 columns]

In [8]:
maxsub.loc['shoes']

0
category item sub               
로퍼       로퍼   detail          버클
              color          브라운
              texture       스웨이드
              heel_height    로우힐
              toe_type      라운드토
...                          ...
펌프스      펌프스  strap        False
              heel_height    하이힐
              toe_type      포인티토
              heel_shape   스틸레토굽
              sole_type      플랫폼

[81 rows x 1 columns]

In [ ]:
pd.pivot_table(dfdf, values='count', index=['type','category','item','sub'], columns=['val'], aggfunc='count')

In [ ]:
df_pos.groupby(by=['type','category','item','color']).count()

In [ ]:
flds = [
    'color','detail','pattern','texture','look','length','sleeve_length',
    'neckline','fit','shape','heel_height','heel_shape','toe_type','sole_type',
    'strap','size','main_material','sub_material'
]

In [ ]:
%%time
out = {}
target_fld = 'color'
groupby_flds = ['type','category','item'] + [target_fld]
value_flds = groupby_flds + ['id']

_df = df_pos[value_flds].groupby(by=groupby_flds).count() - df_neg[value_flds].groupby(by=groupby_flds).count()
_df = _df.dropna()
_df[_df.id>0]

In [ ]:
%%time
_df1 = pd.pivot_table(df_pos, values='id', index=['type','category','item'], columns='color', aggfunc='count')
_df2 = pd.pivot_table(df_neg, values='id', index=['type','category','item'], columns='color', aggfunc='count')
_df1 - _df2

In [ ]:
sum(count.values(), Counter())